In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya
normalizer = malaya.normalize.normalizer(date = False)

In [3]:
import requests

def translate(text):
    r = requests.post('http://localhost:8999/api', timeout=5, json={
        'text': text,
        'from': 'ms',
        'to': 'en',
        'lite': True,
    })
    r = r.json()
    return r

In [4]:
# !rm filtered-dumping-cleaned-common-crawl.txt

In [5]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-cleaned-common-crawl.txt

In [6]:
!wc -l filtered-dumping-cleaned-common-crawl.txt

65082254 filtered-dumping-cleaned-common-crawl.txt


In [63]:
selected = set(['aku', 'kau', 'tak nak', 'taknak', 'tknk', 'aq', 'awk', 'hg', 'harini', 'jugak',
               'jer', 'bodoh', 'lembab', 'hotak', 'bangang', 'bengap', 'kepala otak', 'diorang'])

selected = set(['promo', 'promosi', 'promotion', 'kampeni', 'tau', 'kroni', 'menyorok',
               'ipaq', 'awek', 'balaq', 'xkan', 'takkan', 'macamana', 'lepaih', 'akai', 'perabih',
               'depa', 'sapa', 'kecek', 'pakwe', 'mimin'])

selected = set(['bodo', 'bengap', 'mangkuk', 'sial', 'boroi', 'lepak', 'bengong', 'bingung',
               'kje', 'keje', 'pun', 'punn', 'mane', 'ade', 'sapa', 'kawen',
               'aku', 'kau', 'tak nak', 'taknak', 'tknk', 'aq', 'awk', 'hg', 'harini', 'jugak',
               'jer', 'bodoh', 'lembab', 'hotak', 'bangang', 'bengap', 'kepala otak', 'diorang',
               'promo', 'promosi', 'promotion', 'kampeni', 'tau', 'kroni', 'menyorok',
               'ipaq', 'awek', 'balaq', 'xkan', 'takkan', 'macamana', 'lepaih', 'akai', 'perabih',
               'depa', 'sapa', 'kecek', 'pakwe', 'mimin', 'xpe', 'saye', 'jer', 'pandai', 'cekang',
               'dlm', 'dtg', 'cintai', 'syg', 'pakwa', 'makwa', 'laki', 'pmpuan', 'perempuan',
               'habaq', 'teloq', 'sedih', 'gembira', 'cerdik', 'bongok', 'pandai', 'melatah',
               'kepala', 'lutut', 'kangkang', 'kaki', 'xpa', 'xper', 'xpaa', 'xpaham', 'xfaham',
               'faham', 'lah', 'gak', 'ayaq', 'air'])

rejected = set(['bogel', 'comel', 'porn', 'porno', 'hisap', 'main', 'pantat', 'batang', 'kencing'])

In [8]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [9]:
from tqdm import tqdm
from unidecode import unidecode
import re

def cleaning(s):
    s = s.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    return re.sub(r'[ ]+', ' ', unidecode(s)).strip()

def remove(s):
    s = re.sub('[^A-Za-z ]+', ' ', s)
    return re.sub(r'[ ]+', ' ', s.lower()).strip()

def fix_spacing_punct(string):
    string = string.replace(' , ', ', ').replace(' !', '!').replace(' . ', '. ').replace('( ', '(') \
    .replace(' )', ')')
    return re.sub(r'[ ]+', ' ', string).strip()

In [10]:
f = []
already = set()
with open('filtered-dumping-cleaned-common-crawl.txt') as fopen:
    for l in tqdm(fopen):
        l = l.strip()
        if not len(l):
            continue
            
        if l.isupper():
            continue
            
        if len(l.split()) > 100:
            continue
            
        if len(l.split()) < 2:
            continue
        
        f.append(l)

65082254it [00:36, 1799276.52it/s]


In [11]:
import mp

In [12]:
def loop(texts):
    texts, _ = texts
    f = []
    already = set()
    for l in tqdm(texts):
    
        lower = remove(l)
        splitted = set(lower.split())
        
        if len(splitted & rejected):
            continue
        
        if lower not in already:
            f.append(cleaning(l))
            already.add(lower)
            
    return f

In [13]:
r_ = mp.multiprocessing(f, loop)

100%|█████████████████████████████| 6493409/6493409 [00:32<00:00, 197886.64it/s]


In [14]:
len(f), len(r_)

(38960459, 12873088)

In [64]:
def loop(texts):
    texts, _ = texts
    r = []
    for l in tqdm(texts):
        lower = remove(l)
        
        if any([len(w) > 30 for w in lower.split()]):
            continue
        
        splitted = set(lower.split())
        
        if len(splitted & selected):
            
            l = cleaning(l)
            
            try:
                normalized = normalizer.normalize(
                    l, normalize_hingga = False, normalize_cardinal = False,
                    normalize_ordinal = False, normalize_pada_hari_bulan = False,
                    normalize_fraction = False, normalize_money = False, normalize_date = False,
                    normalize_time = False, normalize_ic = False,
                    normalize_units = False,
                    normalize_url = False, normalize_percent=False, normalize_telephone = False,
                    not_a_word_threshold = 1e-9,
                )['normalize']

                normalized = fix_spacing_punct(normalized)

            except Exception as e:
                print(e)
                normalized = l
                
            # normalized = l
            
            r.append({'original': l, 'normalized': normalized})
    
    return r

In [65]:
# loop((r_[:10000],0))

In [66]:
r__ = mp.multiprocessing(r_, loop, cores = 12)

100%|███████████████████████████████| 1072757/1072757 [01:47<00:00, 9950.55it/s]


In [67]:
len(r__)

958756

In [72]:
import json

with open('filtered-common-crawl-v3.jsonl', 'w') as fopen:
    for d in tqdm(r__):
        fopen.write(f'{json.dumps(d)}\n')

100%|███████████████████████████████| 958756/958756 [00:01<00:00, 531779.46it/s]


In [73]:
!split -l 50000 -d --additional-suffix=.splitted filtered-common-crawl-v3.jsonl filtered-common-crawl-v3.jsonl

In [70]:
958756 // 50000

19